In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# variables used in script

# data locatie als het notebook voor de eerste keer wordt opgestart is 'notebooks'
data_location = '../data/raw/'

if 'raw' not in os.getcwd():
    os.chdir(data_location)

In [3]:
# show all columns in the dataframe
pd.set_option('max_columns', None)

### Dataset Enexis decentrale opwekkingsgegevens zon-pv  2020 en 2021 (1/1 + 1/7)
https://www.enexis.nl/over-ons/wat-bieden-we/andere-diensten/open-data 

Beschrijving:
https://opendatasitecore.blob.core.windows.net/opendata/toelichting/Handleiding%20Decentrale%20Opwek%20gegevens%20versie%201.0_def.pdf

Halfjaarlijkse update. Momenteel zijn 3 files beschikbaar.
Alleen de aangemelde zonne opwekinstallaties zijn meegeteld. Naar schatting zijn er 10-15% niet aangemeeld. Deze er eventueel aan toe voegen?

| Kolom | Omschrijving |
| ----- | ------------ |
| Peildatum | Peildatum in JJJJMM-formaat (bv 202001) van de decentrale opwekgegevens.
| Netbeheerder | Naam van de netbeheerder, in ons geval Enexis Netbeheer.
| Provincie | Provincie waar de aansluiting met decentrale opwek is aangesloten op ons netwerk.
| Gemeente | Gemeente waar de aansluiting met decentrale opwek is aangesloten op ons netwerk.
| CBS Buurt | De CBS-buurtnaam volgens de meest actuele buurttabel van het CBS. Bestaande datasets worden niet bijgewerkt aan de hand van een nieuw gepubliceerde CBS-buurttabel.
| CBS Buurtcode | De unieke code waarmee het CBS de buurt aanduidt.
| Aantal aansluitingen in CBS-buurt | Het aantal actieve aansluitingen in de betreffende CBS-buurt.
| Aantal aansluitingen met opwekinstallatie | Het aantal aansluitingen dat energie opwekt in de betreffende CBS-buurt. Het gaat om actieve aansluitingen met aangemelde opwekinstallaties.
| Opgesteld vermogen | Totaal opgegeven opgesteld vermogen in kWp van de geregistreerde opwekinstallaties.

In [4]:
def load_data(files):
    df = pd.DataFrame()
    for j in files:
        print('load file: ' + j)
        appenddata = pd.read_csv(j, sep=';', thousands='.', decimal=',',  encoding= 'utf-8')
        # column names hebben soms trailing spaces maar niet in beide files
        appenddata.columns = appenddata.columns.str.rstrip()
        df = df.append(appenddata,ignore_index=True)
    return df    

In [5]:
# file names
# net weer een derde file aan toegevoegd van 1-1-2021
files = ['https://opendatasitecore.blob.core.windows.net/opendata/KV_opwek/Enexis_decentrale_opwek_kv_(zon_pv)_01012020.csv'
         ,'https://opendatasitecore.blob.core.windows.net/opendata/KV_opwek/Enexis_decentrale_opwek_kv_(zon_pv)_01072020.csv'
         ,'https://opendatasitecore.blob.core.windows.net/opendata/KV_opwek/Enexis_decentrale_opwek_kv_(zon_pv)_01012021.csv'
        ]

In [6]:
df = load_data(files)

load file: https://opendatasitecore.blob.core.windows.net/opendata/KV_opwek/Enexis_decentrale_opwek_kv_(zon_pv)_01012020.csv
load file: https://opendatasitecore.blob.core.windows.net/opendata/KV_opwek/Enexis_decentrale_opwek_kv_(zon_pv)_01072020.csv
load file: https://opendatasitecore.blob.core.windows.net/opendata/KV_opwek/Enexis_decentrale_opwek_kv_(zon_pv)_01012021.csv


In [7]:
df.columns

Index(['Peildatum', 'Netbeheerder', 'Provincie', 'Gemeente', 'CBS Buurt',
       'CBS Buurtcode', 'Aantal aansluitingen in CBS-buurt',
       'Aantal aansluitingen met opwekinstallatie', 'Opgesteld vermogen',
       'Unnamed: 9'],
      dtype='object')

In [8]:
# check missing values
missing_total = df.isna().sum().sort_values(ascending=False)
print('Number of attributes with Null values: ' + str(len(missing_total[missing_total > 0])))
print(missing_total[missing_total > 0])

Number of attributes with Null values: 10
Unnamed: 9                                   12675
CBS Buurtcode                                  990
CBS Buurt                                      962
Gemeente                                       575
Provincie                                      572
Aantal aansluitingen in CBS-buurt              570
Peildatum                                      569
Netbeheerder                                   569
Aantal aansluitingen met opwekinstallatie      569
Opgesteld vermogen                             569
dtype: int64


In [9]:
# vreemde kolom aan het eind
df = df.drop(columns=['Unnamed: 9'])

In [10]:
# drop alle rows waar alle columns leeg zijn
df = df.dropna(how='all') 
#df = df.dropna(subset=['Peildatum'])

### Missing values
CBS Buurtcode: 393 missing, verwijderen? Of nog iets mee doen obv gemeente?

In [11]:
df[df['CBS Buurtcode'].isna()]
#df[df['CBS Buurtcode'].isna()]['Opgesteld vermogen'].sum()

,Peildatum,Netbeheerder,Provincie,Gemeente,CBS Buurt,CBS Buurtcode,Aantal aansluitingen in CBS-buurt,Aantal aansluitingen met opwekinstallatie,Opgesteld vermogen
3729,202001.0,Enexis,Drenthe,Aa en Hunze,NaN,NaN,280.0,73.0,651.0
3730,202001.0,Enexis,Drenthe,Assen,NaN,NaN,2499.0,400.0,2464.0
3731,202001.0,Enexis,Drenthe,Borger-Odoorn,NaN,NaN,385.0,73.0,652.0
3732,202001.0,Enexis,Drenthe,Coevorden,NaN,NaN,827.0,125.0,1227.0
3733,202001.0,Enexis,Drenthe,De Wolden,NaN,NaN,265.0,39.0,184.0
...,...,...,...,...,...,...,...,...,...
11978,202101.0,Enexis,Overijssel,Twenterand,NaN,NaN,150.0,16.0,199.0
12005,202101.0,Enexis,Overijssel,Wierden,NaN,NaN,234.0,112.0,460.0
12035,202101.0,Enexis,Overijssel,Zwartewaterland,NaN,NaN,531.0,128.0,506.0
12104,202101.0,Enexis,Overijssel,Zwolle,NaN,NaN,2195.0,665.0,2373.0


In [12]:
# fill missing values 
df['CBS Buurt'] = df['CBS Buurt'].fillna('Onbekend')
df['Gemeente'] = df['Gemeente'].fillna('Onbekend')
df['Provincie'] = df['Provincie'].fillna('Onbekend')
df['Aantal aansluitingen in CBS-buurt'] = df['Aantal aansluitingen in CBS-buurt'].fillna(0)
df['CBS Buurtcode'] = df['CBS Buurtcode'].fillna(9999)

In [13]:
# select numeric columns
numeric_columns = df.select_dtypes(include=['number']).columns  #To select all *numeric* types, use ``np.number`` or ``'number'``
#numeric_columns
# change format to integer for numeric columns
df[numeric_columns]= df[numeric_columns].astype(int)

In [14]:
# peildatum als datum (yyyy-mm)
df['Peildatum'] = pd.to_datetime(df['Peildatum'].astype(str).str[0:6], format='%Y%m', errors='ignore')

In [28]:
df.head()

,Peildatum,Netbeheerder,Provincie,Gemeente,CBS Buurt,CBS Buurtcode,Aantal aansluitingen in CBS-buurt,Aantal aansluitingen met opwekinstallatie,Opgesteld vermogen,PC4
0,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-Centrum,30000,1480,155,507,9901
1,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-West,30001,1331,479,1685,9903
2,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-Oost,30002,2826,439,1616,9902
3,2020-01-01,Enexis,Groningen,Appingedam,Verspreide huizen Damsterdiep en Eemskanaal,30007,230,97,405,9902
4,2020-01-01,Enexis,Groningen,Appingedam,Verspreide huizen ten zuiden van Eemskanaal,30008,102,21,61,9902


In [15]:
# data locatie om de ht5 file op te slaan
data_processed_location = '../processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

In [16]:
try:
    os.remove('decentrale_opwekkingsgegevens_data.h5')
except:
    print ('File nog in gebruik of niet gevonden')

store = pd.HDFStore('decentrale_opwekkingsgegevens_data.h5')
store['decentrale_opwekkingsgegevens_data'] = df
store.close()

# CBS Buurtcode koppelen aan PC4

In [16]:
file = 'https://github.com/processminery/jads_enexis/blob/main/data/pc6-gwb2020.csv?raw=true'

In [17]:
df_brt = pd.read_csv(file, sep=';', thousands='.', decimal=',',  encoding= 'utf-8')

In [18]:
df_brt.head()

,PC6,Buurt2020,Wijk2020,Gemeente2020
0,1011AB,3630400,36304,363
1,1011AC,3630400,36304,363
2,1011AD,3630400,36304,363
3,1011AE,3630400,36304,363
4,1011AG,3630403,36304,363


In [19]:
# 4 cijferige postcode uit PC6 halen
df_brt['PC4'] = df_brt['PC6'].astype(str).str[0:4]

In [20]:
df_brt.head()

,PC6,Buurt2020,Wijk2020,Gemeente2020,PC4
0,1011AB,3630400,36304,363,1011
1,1011AC,3630400,36304,363,1011
2,1011AD,3630400,36304,363,1011
3,1011AE,3630400,36304,363,1011
4,1011AG,3630403,36304,363,1011


In [21]:
# meest voorkomende PC4 per buurtcode
df2 = df_brt.groupby(['Buurt2020'])['PC4'].agg(lambda x:x.value_counts().index[0])

In [22]:
df2.head()

Buurt2020
30000    9901
30001    9903
30002    9902
30007    9902
30008    9902
Name: PC4, dtype: object

In [23]:
# join df met df2
df = pd.merge(df, df2, left_on='CBS Buurtcode', right_on='Buurt2020', how='left', suffixes=('','_2'))

In [24]:
df.head()

,Peildatum,Netbeheerder,Provincie,Gemeente,CBS Buurt,CBS Buurtcode,Aantal aansluitingen in CBS-buurt,Aantal aansluitingen met opwekinstallatie,Opgesteld vermogen,PC4
0,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-Centrum,30000,1480,155,507,9901
1,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-West,30001,1331,479,1685,9903
2,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-Oost,30002,2826,439,1616,9902
3,2020-01-01,Enexis,Groningen,Appingedam,Verspreide huizen Damsterdiep en Eemskanaal,30007,230,97,405,9902
4,2020-01-01,Enexis,Groningen,Appingedam,Verspreide huizen ten zuiden van Eemskanaal,30008,102,21,61,9902


In [25]:
#groupby PC4 en peildatum
df_pc4 = df.groupby(['PC4','Peildatum'], as_index=False).agg({'Aantal aansluitingen in CBS-buurt':'sum'
                                                          , 'Aantal aansluitingen met opwekinstallatie':'sum'
                                                          , 'Opgesteld vermogen':'sum'})

In [26]:
# extra attributen toevoegen:
# fractie aansluitingen met opwekinstallatie
# vermogen per opwekinstallatie in kWp
df_pc4['Fractie met Opwerkinstallatie'] = (df_pc4['Aantal aansluitingen met opwekinstallatie'] / df_pc4['Aantal aansluitingen in CBS-buurt']).round(2)
df_pc4['Vermogen per Opwekinstallatie in kWp'] = (df_pc4['Opgesteld vermogen'] / df_pc4['Aantal aansluitingen met opwekinstallatie']).round(1)

In [27]:
df_pc4.head()

,PC4,Peildatum,Aantal aansluitingen in CBS-buurt,Aantal aansluitingen met opwekinstallatie,Opgesteld vermogen,Fractie met Opwerkinstallatie,Vermogen per Opwekinstallatie in kWp
0,4251,2020-01-01,1922,117,335,0.06,2.9
1,4261,2020-01-01,289,39,197,0.13,5.1
2,4264,2020-01-01,1179,76,383,0.06,5.0
3,4265,2020-01-01,615,79,458,0.13,5.8
4,4266,2020-01-01,239,34,119,0.14,3.5
